<a href="https://colab.research.google.com/github/Jibrangit/DL_Assignments/blob/HW6/Copy_of_HW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
import numpy as np
x_load = np.load('/content/drive/MyDrive/Colab Notebooks/faces.npy')
y_load = np.load('/content/drive/MyDrive/Colab Notebooks/ages.npy')

print(np.shape(x_load))
no_data = int(x_load.shape[0])
print(no_data)

np.random.seed(0)
X_rgb = np.random.randn(7500, 48, 48, 3)
for i in range(0, len(x_load)):
  img = x_load[i, :, :]
  stacked_img = np.stack((img,)*3, axis=-1)
  X_rgb[i, :, :, 0] = stacked_img[:, :, 0]
  X_rgb[i, :, :, 1] = stacked_img[:, :, 1]
  X_rgb[i, :, :, 2] = stacked_img[:, :, 2]

x_load = X_rgb
print(np.shape(x_load))

(7500, 48, 48)
7500
(7500, 48, 48, 3)


In [22]:
import random
def random_rearrange (X_tr, y_tr, seed):
    np.random.seed(seed)
    np.random.shuffle(X_tr)
    np.random.seed(seed)
    np.random.shuffle(y_tr)

no_data = int(x_load.shape[0])
train_size = int(0.8*0.8*no_data)
test_size = int(0.2*no_data)

print(y_load[0])   

random_rearrange(x_load, y_load, 0)
print(y_load[0])                        #Just checking if shuffling ahs been done coorrectly.

(x_test, x_train, x_valid) = x_load[:test_size], x_load[test_size:test_size+train_size],  x_load[test_size+train_size:]
(y_test, y_train, y_valid) = y_load[:test_size], y_load[test_size:test_size+train_size], y_load[test_size+train_size:]

w, h = 48, 48

# x_train = x_train.astype('float32') / 255
# x_valid = x_valid.astype('float32') / 255
# x_test = x_test.astype('float32') / 255

# x_train = x_train.reshape(x_train.shape[0], w, h, 1)
# print(np.shape(x_train))
# x_valid = x_valid.reshape(x_valid.shape[0], w, h, 1)
# x_test = x_test.reshape(x_test.shape[0], w, h, 1)




40.0
48.0


In [23]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten

In [24]:
model = Sequential()

In [25]:
model.add(Conv2D(input_shape=(48,48,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=1, activation="relu"))

##units are 1 because we are using binary activation

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 48, 48, 64)        1792      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 48, 48, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 24, 24, 128)       73856     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 24, 24, 128)       147584    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 12, 12, 256)      

In [30]:
import tensorflow as tf

from keras.metrics import RootMeanSquaredError
import keras.backend as K
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 
        print(y_true)
model.compile(
  loss=root_mean_squared_error,
  optimizer='RMSProp',
  metrics=[RootMeanSquaredError()]
)

In [ ]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose = 1, save_best_only=True)
model.fit(x_train,
          y_train,
          batch_size=8,
          epochs= 10,
          validation_data=(x_valid, y_valid),
          callbacks=[checkpointer])



Epoch 1/10
600/600 [==============================] - 24s 37ms/step - loss: 42.0344 - root_mean_squared_error: 42.3028 - val_loss: 42.0475 - val_root_mean_squared_error: 42.3567

Epoch 00001: val_loss improved from inf to 42.04753, saving model to model.weights.best.hdf5
Epoch 2/10
600/600 [==============================] - 23s 38ms/step - loss: 41.7890 - root_mean_squared_error: 42.1035 - val_loss: 42.0475 - val_root_mean_squared_error: 42.3567

Epoch 00002: val_loss did not improve from 42.04753
Epoch 3/10
600/600 [==============================] - 22s 37ms/step - loss: 41.8295 - root_mean_squared_error: 42.2329 - val_loss: 42.0475 - val_root_mean_squared_error: 42.3567

Epoch 00003: val_loss did not improve from 42.04753
Epoch 4/10
545/600 [==========================>...] - ETA: 1s - loss: 42.1016 - root_mean_squared_error: 42.4456

In [15]:
### prediction code
# Load the weights with the best validation accuracy
model.load_weights('model.weights.best.hdf5')
# Evaluate the model on test set
score = model.evaluate(x_test, y_test, verbose=0)
# Print test accuracy
print('\n', 'Test accuracy:', score[1])
y_hat = model.predict(x_test)
print(y_hat)
print(y_test[55])


 Test accuracy: 0.006666666828095913
[[40.08122 ]
 [40.081203]
 [40.08123 ]
 ...
 [40.08132 ]
 [40.081207]
 [40.08121 ]]
28.0


In [11]:
import numpy as np
img = np.array([[1, 2], [3, 4]])
print(np.shape(img))
stacked_img = np.stack((img,)*3, axis=-1)
print(np.shape(stacked_img))
print("=============")
np.random.seed(0)
x_train = np.random.randn(48000, 48, 48)





(2, 2)
(2, 2, 3)


In [12]:
print(X_rgb.shape)

(7500, 48, 48, 3)


In [13]:
print(X_rgb[77, :, :, 2])

[[ 35.  58.  81. ... 138. 132. 125.]
 [ 36.  58.  82. ... 139. 134. 130.]
 [ 40.  55.  80. ... 144. 142. 140.]
 ...
 [ 71.  74.  79. ... 149. 132. 116.]
 [ 71.  76.  80. ... 139. 121. 114.]
 [ 69.  74.  79. ... 126. 112. 116.]]
